In [29]:
import numpy as np
import sympy

import plotly
import plotly.graph_objects as go
import plotly.express as px

In [40]:
def curvature_discrete_parametric(X: np.array, Y: np.array, Z: np.array):
    # known by MATLAB as surfature
    # where X, Y, Z matrices have a shape (lr+1,lb+1)
    # https://github.com/sujithTSR/surface-curvature/blob/master/surface.py

    (lr, lb) = X.shape

    # First Derivatives
    Xv, Xu = np.gradient(X)
    Yv, Yu = np.gradient(Y)
    Zv, Zu = np.gradient(Z)

    # Second Derivatives
    Xuv, Xuu = np.gradient(Xu)
    Yuv, Yuu = np.gradient(Yu)
    Zuv, Zuu = np.gradient(Zu)

    Xvv, Xuv = np.gradient(Xv)
    Yvv, Yuv = np.gradient(Yv)
    Zvv, Zuv = np.gradient(Zv)

    # Reshape to 1D vectors (same as ravel?)
    Xu = np.reshape(Xu, lr * lb)
    Yu = np.reshape(Yu, lr * lb)
    Zu = np.reshape(Zu, lr * lb)
    Xv = np.reshape(Xv, lr * lb)
    Yv = np.reshape(Yv, lr * lb)
    Zv = np.reshape(Zv, lr * lb)
    Xuu = np.reshape(Xuu, lr * lb)
    Yuu = np.reshape(Yuu, lr * lb)
    Zuu = np.reshape(Zuu, lr * lb)
    Xuv = np.reshape(Xuv, lr * lb)
    Yuv = np.reshape(Yuv, lr * lb)
    Zuv = np.reshape(Zuv, lr * lb)
    Xvv = np.reshape(Xvv, lr * lb)
    Yvv = np.reshape(Yvv, lr * lb)
    Zvv = np.reshape(Zvv, lr * lb)

    # Xu and Xv are now vectors
    Xu = np.c_[Xu, Yu, Zu]
    Xv = np.c_[Xv, Yv, Zv]
    Xuu = np.c_[Xuu, Yuu, Zuu]
    Xuv = np.c_[Xuv, Yuv, Zuv]
    Xvv = np.c_[Xvv, Yvv, Zvv]

    # local parametrization function unit normal vector
    m = np.cross(Xu, Xv, axisa=1, axisb=1)
    print(f'm.shape = {m.shape}')
    print(f'm = {m}')
    p = np.sqrt(np.einsum("ij,ij->i", m, m))
    n = m / np.c_[p, p, p]
    print(f'n.shape = {n.shape}')
    print(f'n = {n}')


    # First fundamental Coefficients of the surface (E,F,G)
    E = np.einsum("ij,ij->i", Xu, Xu) # element-wise multiplication Xu * Xu
    F = np.einsum("ij,ij->i", Xu, Xv)
    G = np.einsum("ij,ij->i", Xv, Xv)

    # Second fundamental Coefficients of the surface (L,M,N)
    L = np.einsum("ij,ij->i", Xuu, n) # element-wise multiplication Xuu * n
    M = np.einsum("ij,ij->i", Xuv, n)
    N = np.einsum("ij,ij->i", Xvv, n)

    # % Gaussian Curvature
    K = (L * N - M**2) / (E * G - F**2)

    # % Mean Curvature
    H = (E * N + G * L - 2 * F * M) / (2 * (E * G - F**2))

    ## % Shape Operator as 3D a matrix of 2D matrices (2, 2, lr*lb)
    ## this method is giving wrong directions and values
    # LMMN = np.array([[L, M], [M, N]])
    # EFFG = np.array([[E, F], [F, G]])
    #print(f'LMMN.shape = {LMMN.shape}')
    #print(f'EFFG.shape = {EFFG.shape}')
    #print(f'LMMN = {LMMN}')
    ## reshape so that the 2D matrices are in the last dimension (lr*lb, 2, 2)
    # LMMN = np.swapaxes(LMMN, 0, 2)
    # LMMN = np.swapaxes(LMMN, 1, 2)
    # EFFG = np.swapaxes(EFFG, 0, 2)
    # EFFG = np.swapaxes(EFFG, 1, 2)
    # print(f'LMMN = {LMMN}')
    # P = LMMN * np.linalg.inv(EFFG) # TODO: error is happening in P somewhere

    # this method works, it is proven bu using it in discrete_shape_monge.ipynb
    P = np.array([[L*G - M*F, M*E-L*F], [M*E-L*F, N*E-M*F]])
    P = P / (E*G-F*F)
    P = np.swapaxes(P, 0, 2)
    P = np.swapaxes(P, 1, 2)


    print(f'P.shape = {P.shape}')
    print(f'P = {P}')
    X = np.linalg.eig(P)

    # the result of eig is a tuple of (eigenvalues, eigenvectors)
    print(f'X[1] {X[1].shape} {X[1]}')
    k1 = X[0][:, 0]  # all the first eigenvalues
    k2 = X[0][:, 1]  # all the second eigenvalues
    X1 = X[1][:, :, 0]  # all the first eigenvectors
    X2 = X[1][:, :, 1]  # all the second eigenvectors

    X1 = np.expand_dims(X1, 2)  # add a dimension to the end (lr*lb, 2, 1)
    X2 = np.expand_dims(X2, 2)  # add a dimension to the end (lr*lb, 2, 1)
    print(f'X1.shape = {X1.shape}')
    print(f'X2.shape = {X2.shape}')

    dX = np.dstack((Xu, Xv))
    print(f'Xu.shape = {Xu.shape}')
    print(f'Xv.shape = {Xv.shape}')
    # print(F'Yu.shape = {Yu.shape}')
    # print(f'Yv.shape = {Yv.shape}')
    # print(f'Zu.shape = {Zu.shape}')
    # print(f'Zv.shape = {Zv.shape}')
    # dX = np.swapaxes(dX, 0, 2)
    # dX = np.swapaxes(dX, 2, 1)
    print(f'dX.shape = {dX.shape}')
    print(f'dX = {dX}')

    # matrix multiplication of dX and X for each point
    # k1vec = np.einsum("ijk,ikl->ilj", dX, X1)
    # k2vec = np.einsum("ijk,ikl->ilj", dX, X2)
    # matrix multiplication of dX and X for each point
    k1vec = np.einsum("lij,ljk->lik", dX, X1)
    k2vec = np.einsum("lij,ljk->lik", dX, X2)

    # normalize the vectors
    k1vec = k1vec / np.linalg.norm(k1vec, axis=2, keepdims=True)
    k2vec = k2vec / np.linalg.norm(k2vec, axis=2, keepdims=True)

    # #% Principal Curvatures k1, k2 (alternative from gaussian and mean curvature)
    # k1 = H + np.sqrt(H**2 - K)
    # k2 = H - np.sqrt(H**2 - K)

    # reshape back to 2D x,y matrices
    K = np.reshape(K, (lr, lb))
    H = np.reshape(H, (lr, lb))
    k1 = np.reshape(k1, (lr, lb))
    k2 = np.reshape(k2, (lr, lb))
    k1vec = np.reshape(k1vec, (lr, lb, 3))
    k2vec = np.reshape(k2vec, (lr, lb, 3))

    return K, H, k1, k2, k1vec, k2vec

In [31]:
u, v = sympy.symbols("u v")
# f_explicit = sympy.sqrt(100 - v**2 - u**2)
# f_explicit = sympy.sqrt(100 - v**2 -2*u**2)
# f_explicit = sympy.sqrt(100 - (0.5*v - 0.5*u))
f_explicit = 2 + -(2*u-v)*(2*u-v)
x, y = u, v

# coordinate range
xx = np.linspace(-1, 1, 21)
yy = np.linspace(-1, 1, 21)

# make coordinate point
X, Y = np.meshgrid(xx, yy)

# dependent variable point on coordinate
f2 = sympy.lambdify((x, y), f_explicit)
Z = f2(X, Y)

# print(X)

fig = go.Figure(go.Surface(x=X, y=Y, z=Z, surfacecolor=Z)) # np.zeros_like(Z)
fig.show()

In [43]:
K, H, k1, k2, k1vec, k2vec = curvature_discrete_parametric(X, Y, Z)

print('k1[10,10]', k1[10,10])
print('k2[10,10]', k2[10,10])
print('k1vec[10,10]', k1vec[10,10])
print('k2vec[10,10]', k2vec[10,10])
print('k1vec * k2vec', np.dot(k1vec[10,10], k2vec[10,10]))
print('K[10,10]', K[10,10])
print('H[10,10]', H[10,10])

# print(k1vec[1,1])
# print(k2vec[1,1])
# print(np.dot(k1vec[1,1], k2vec[1,1]))

# fig = px.imshow(k2, labels=dict(x="X", y="Y", color="k2"),)


fig = go.Figure(
    data=[
        go.Cone(
            x=X.ravel(),
            y=Y.ravel(),
            z=Z.ravel(),
            u=k2vec[:, :, 0].ravel(),
            v=k2vec[:, :, 1].ravel(),
            w=k2vec[:, :, 2].ravel(),
            # u=k2vec[:, 0].ravel(),
            # v=k2vec[:, 1].ravel(),
            # w=k2vec[:, 2].ravel(),
            # sizemode="absolute",
            # sizeref=2,
            anchor="tip",
        ),
        go.Cone(
            x=X.ravel(),
            y=Y.ravel(),
            z=Z.ravel(),
            u=k1vec[:, :, 0].ravel(),
            v=k1vec[:, :, 1].ravel(),
            w=k1vec[:, :, 2].ravel(),
            # sizemode="absolute",
            # sizeref=1,
            anchor="tip",
        ),
    ]
)
fig.add_trace(go.Surface(x=X, y=Y, z=Z, surfacecolor=H)) # , surfacecolor=H
fig.show()

m.shape = (441, 3)
m = [[-0.036  0.021  0.01 ]
 [-0.032  0.017  0.01 ]
 [-0.024  0.013  0.01 ]
 ...
 [ 0.024 -0.013  0.01 ]
 [ 0.032 -0.017  0.01 ]
 [ 0.036 -0.021  0.01 ]]
n.shape = (441, 3)
n = [[-0.83993934  0.48996461  0.23331648]
 [-0.85129268  0.45224923  0.26602896]
 [-0.8256251   0.4472136   0.34401046]
 ...
 [ 0.8256251  -0.4472136   0.34401046]
 [ 0.85129268 -0.45224923  0.26602896]
 [ 0.83993934 -0.48996461  0.23331648]]
P.shape = (441, 2, 2)
P = [[[ 0.10922818  0.32514436]
  [ 0.32514436  0.20677149]]

 [[-0.02974705  0.23195165]
  [ 0.23195165  0.19806261]]

 [[-0.36803012  0.0846795 ]
  [ 0.0846795   0.23286862]]

 ...

 [[-0.36803012  0.0846795 ]
  [ 0.0846795   0.23286862]]

 [[-0.02974705  0.23195165]
  [ 0.23195165  0.19806261]]

 [[ 0.10922818  0.32514436]
  [ 0.32514436  0.20677149]]]
X[1] (441, 2, 2) [[[-0.75774021 -0.65255634]
  [ 0.65255634 -0.75774021]]

 [[-0.84876104 -0.5287766 ]
  [ 0.5287766  -0.84876104]]

 [[-0.99058116 -0.13692684]
  [ 0.13692684 -0.99058

C:\Users\a0487637\AppData\Local\Temp\ipykernel_13304\2037839180.py:130: RuntimeWarning:

invalid value encountered in divide

C:\Users\a0487637\AppData\Local\Temp\ipykernel_13304\2037839180.py:131: RuntimeWarning:

invalid value encountered in divide



In [33]:
a = np.array([[[1,1,1], [2,2,2]]])
a.shape # (1, 3)
# (lr, lb) = a.shape
# a = np.reshape(a, (1,1, lr * lb))
# a.shape # (1, 3)
# c = np.swapaxes(a, 0, 2)
# c
# c.shape
# a

(1, 2, 3)